In [3]:
from pycaret.datasets import get_data 
from pycaret.classification import *
from pycaret.utils import check_metric

In [4]:
# dataset 생성
dataset = get_data('juice')

,Id,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


In [12]:
dataset.to_csv('dataset.csv',index = False)

In [5]:
# setup으로 init
clf = setup(data=dataset, target='Purchase', use_gpu=True)

,Description,Value
0,session_id,4989
1,Target,Purchase
2,Target Type,Binary
3,Label Encoded,"CH: 0, MM: 1"
4,Original Data,"(1070, 19)"
5,Missing Values,False
6,Numeric Features,13
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


In [6]:
# ML 모델 성능 비교
top5 = compare_models(sort='Accuracy', n_select=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.8302,0.8868,0.7522,0.8018,0.7746,0.6388,0.6412,0.0110
lr,Logistic Regression,0.8289,0.8877,0.7283,0.8157,0.7676,0.6330,0.6373,0.1110
ridge,Ridge Classifier,0.8248,0.0000,0.7420,0.7952,0.7665,0.6267,0.6288,0.0080
gbc,Gradient Boosting Classifier,0.8128,0.8785,0.7424,0.7730,0.7547,0.6038,0.6068,0.0950
lightgbm,Light Gradient Boosting Machine,0.7955,0.8625,0.7249,0.7459,0.7337,0.5680,0.5698,0.9490
ada,Ada Boost Classifier,0.7914,0.8666,0.6941,0.7574,0.7209,0.5553,0.5597,0.0900
rf,Random Forest Classifier,0.7768,0.8513,0.6978,0.7231,0.7078,0.5277,0.5303,0.4410
nb,Naive Bayes,0.7674,0.8295,0.7939,0.6708,0.7259,0.5266,0.5339,0.0100
dt,Decision Tree Classifier,0.7621,0.7579,0.6872,0.7021,0.6915,0.4985,0.5017,0.0100
et,Extra Trees Classifier,0.7474,0.8128,0.6634,0.6809,0.6704,0.4660,0.4675,0.4510


In [8]:
plot_model(top5)

ValueError: Estimator [LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001), LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=4989, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=4989,
                solver='auto', tol=0.001), GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
      

In [6]:
# 모델 튜닝, blending
tuned_top5 = [tune_model(i) for i in top5]
blender_top5 = blend_models(estimator_list=tuned_top5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7600,0.0000,0.6071,0.7083,0.6538,0.4718,0.4751
1,0.8400,0.0000,0.7857,0.7857,0.7857,0.6581,0.6581
2,0.8933,0.0000,0.8214,0.8846,0.8519,0.7687,0.7700
3,0.8400,0.0000,0.7500,0.8077,0.7778,0.6530,0.6541
4,0.8400,0.0000,0.7500,0.8077,0.7778,0.6530,0.6541
5,0.8800,0.0000,0.7857,0.8800,0.8302,0.7379,0.7407
6,0.7867,0.0000,0.7143,0.7143,0.7143,0.5441,0.5441
7,0.8533,0.0000,0.8214,0.7931,0.8070,0.6888,0.6891
8,0.8514,0.0000,0.8148,0.7857,0.8000,0.6818,0.6821


In [7]:
# 모델 최종 확정, 전체 데이터 학습
final_model = finalize_model(blender_top5)

In [8]:
# 테스트셋 predict
prediction = predict_model(final_model, data=dataset.iloc[-100:])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0,0.7917,0,0,0,0,0


In [9]:
# metric 확인
check_metric(prediction['Purchase'], prediction['Label'], metric = 'Accuracy')

0.81

In [13]:
prediction.to_csv('prediction.csv',index = False)